# Загрузка данных в Apache Impala

## Подгтовка

### Подготовка HDFS директории

In [ ]:
hdfs dfs -mkdir -p /user/hadoop
hdfs dfs -chmod -R 777 /user/hadoop
hdfs dfs -ls /user

### Создание внешних директорий

In [ ]:
HOST=${IMPALA_HOST} execute \
mkdir -p /user/hive/warehouse/external_data/{tab1,tab2}

### Создание файлов

In [ ]:
HOST=${IMPALA_HOST} \
new_file /tmp/tab1.csv <<EOF
1,true,123.123,2012-10-24 08:55:00
2,false,1243.5,2012-10-25 13:40:00
3,false,24453.325,2008-08-22 09:33:21.123
4,false,243423.325,2007-05-12 22:32:21.33454
5,true,243.325,1953-04-22 09:11:33
EOF

HOST=${IMPALA_HOST} execute \
cp /tmp/tab1.csv /user/hive/warehouse/external_data/tab1

In [ ]:
HOST=${IMPALA_HOST} execute \
cat /user/hive/warehouse/external_data/tab1/tab1.csv

In [ ]:
HOST=${IMPALA_HOST} \
new_file /tmp/tab2.csv <<EOF
1,true,12789.123
2,false,1243.5
3,false,24453.325
4,false,2423.3254
5,true,243.325
60,false,243565423.325
70,true,243.325
80,false,243423.325
90,true,243.325
EOF

HOST=${IMPALA_HOST} execute \
cp /tmp/tab2.csv /user/hive/warehouse/external_data/tab2

In [ ]:
HOST=${IMPALA_HOST} execute \
cat /user/hive/warehouse/external_data/tab2/tab2.csv

### Создание базы данных

In [ ]:
impala-shell -i "${IMPALA_HOST}" -q "
DROP DATABASE IF EXISTS loader_demo_db
CASCADE
"

In [ ]:
impala-shell -i "${IMPALA_HOST}" -q "
CREATE DATABASE loader_demo_db
"

## Загрузка данных через внешнюю таблицу

Инструкция `EXTERNAL` означает, что данные находятся за пределами ожидамой Impala директории с данными. Таблицы созданные с инструкцией `EXTERNAL` не владеют файлами: при удалении таблицы, файлы сохраняются на файловой системе.

Путь до директории с данными задается через инструкцию `LOCATION`.

In [ ]:
impala-shell -i "${IMPALA_HOST}" -d loader_demo_db -q "
DROP TABLE IF EXISTS tab1_ext;

CREATE EXTERNAL TABLE tab1_ext (
   id INT,
   col_1 BOOLEAN,
   col_2 DOUBLE,
   col_3 TIMESTAMP
)
ROW FORMAT DELIMITED FIELDS TERMINATED BY ','
STORED AS TEXTFILE
LOCATION '/user/hive/warehouse/external_data/tab1';
"

Таблица `tab2` является внешней таблицей аналогично таблице `tab1`:

In [ ]:
impala-shell -i "${IMPALA_HOST}" -d loader_demo_db -q "
DROP TABLE IF EXISTS tab2_ext;

CREATE EXTERNAL TABLE tab2_ext (
   id INT,
   col_1 BOOLEAN,
   col_2 DOUBLE
)
ROW FORMAT DELIMITED FIELDS TERMINATED BY ','
STORED AS TEXTFILE
LOCATION '/user/hive/warehouse/external_data/tab2';
"

Таблица `tab3` создается без ключевого слова `EXTERNAL`, следовательно:

- `tab3` является управляемой (managed) таблицей,
- все данные сохраняются в директорию базы данных Impala,
- файлы с данными создаются в директории базы данных Impala после добавления данных в таблицу.

In [ ]:
impala-shell -i "${IMPALA_HOST}" -d loader_demo_db -q "
DROP TABLE IF EXISTS tab3_managed;

CREATE TABLE tab3_managed (
   id INT,
   col_1 BOOLEAN,
   col_2 DOUBLE,
   month INT,
   day INT
)
ROW FORMAT DELIMITED FIELDS TERMINATED BY ',';
"

### Описание внешней таблицы

Чтобы понять, является таблица внешней или управляемой, необходимо выполнить `DESCRIBE` команду и найти строку `Table Type`:

In [ ]:
impala-shell -i "${IMPALA_HOST}" -d loader_demo_db -q "
DESCRIBE EXTENDED tab1_ext
"

In [ ]:
impala-shell -i "${IMPALA_HOST}" -d loader_demo_db -q "
DESCRIBE EXTENDED tab3_managed
"

### Получение данных

Для получения данных из внешних таблиц необходимо использовать обычные инструкции `SELECT`:

In [ ]:
impala-shell -i "${IMPALA_HOST}" -d loader_demo_db -q "
SELECT * FROM tab1_ext
"

In [ ]:
impala-shell -i "${IMPALA_HOST}" -d loader_demo_db -q "
SELECT *
  FROM tab2_ext
 LIMIT 5
"

In [ ]:
impala-shell -i "${IMPALA_HOST}" -d loader_demo_db -q "
SELECT t1.col_1
     , COUNT(*)
     , MAX(t2.col_2)
     , MIN(t2.col_2)
  FROM tab2_ext t2
  LEFT JOIN tab1_ext t1
 USING (id)
 GROUP BY col_1
 ORDER BY 1
 LIMIT 5;
"

In [ ]:
impala-shell -i "${IMPALA_HOST}" -d loader_demo_db -q "
SELECT t2.*
  FROM tab2_ext t2,
       (
           SELECT t1.col_1
                , MAX(t2.col_2) AS max_col2
             FROM tab2_ext t2
                , tab1_ext t1
            WHERE t1.id = t2.id
            GROUP BY col_1
       ) subquery1
 WHERE subquery1.max_col2 = t2.col_2;
"

### Загрузка данных в Apache Impala

Для загрузки данных в Apache Impala можно выполнить `INSERT ... SELECT` инструкцию:

In [ ]:
impala-shell -i "${IMPALA_HOST}" -d loader_demo_db -q "
SELECT * FROM tab3_managed
"

In [ ]:
impala-shell -i "${IMPALA_HOST}" -d loader_demo_db -q "
INSERT INTO tab3_managed
SELECT id
     , col_1
     , col_2
     , MONTH(col_3)
     , DAYOFMONTH(col_3)
  FROM tab1_ext
 WHERE YEAR(col_3) = 2012;
"

In [ ]:
impala-shell -i "${IMPALA_HOST}" -d loader_demo_db -q "
SELECT * FROM tab3_managed
"

### Перезапись данных

Перед загрузкой внешних данных в таблицу часто возникает потребность предварительно очистить таблицу.

Традиционно применяется шаблон `TRUNCATE TABLE + INSERT ... SELECT`:

In [ ]:
impala-shell -i "${IMPALA_HOST}" -d loader_demo_db -q "
TRUNCATE TABLE tab3_managed;

INSERT INTO tab3_managed
SELECT id
     , col_1
     , col_2
     , MONTH(col_3)
     , DAYOFMONTH(col_3)
  FROM tab1_ext
 WHERE YEAR(col_3) = 2012;
"

In [ ]:
impala-shell -i "${IMPALA_HOST}" -d loader_demo_db -q "
SELECT * FROM tab3_managed;
"

В Apache Impala имеется более удобная конструкцию для подобных случаев:

In [ ]:
impala-shell -i "${IMPALA_HOST}" -d loader_demo_db -q "
INSERT OVERWRITE TABLE tab3_managed
SELECT id
     , col_1
     , col_2
     , MONTH(col_3)
     , DAYOFMONTH(col_3)
  FROM tab1_ext
 WHERE YEAR(col_3) = 2012;
"

In [ ]:
impala-shell -i "${IMPALA_HOST}" -d loader_demo_db -q "
SELECT * FROM tab3_managed;
"

## Загрузка файлов в управляемые таблицы

### Подготовка управляемой таблицы

In [ ]:
impala-shell -i "${IMPALA_HOST}" -d loader_demo_db -q "
DROP TABLE IF EXISTS tab1_managed;

CREATE TABLE tab1_managed (
   id INT,
   col_1 BOOLEAN,
   col_2 DOUBLE,
   col_3 TIMESTAMP
)
ROW FORMAT DELIMITED
FIELDS TERMINATED BY ','
STORED AS TEXTFILE
"

### Подготовка внешнего файла на HDFS

In [ ]:
HOST=${NAMENODE_HOST:-hadoop} \
new_file /tmp/tab1_hdfs.csv < /tmp/tab1.csv

hdfs dfs -put -f /tmp/tab1_hdfs.csv /user/hadoop/tab1_hdfs.csv

hdfs dfs -cat /user/hadoop/tab1_hdfs.csv

### Загрузка данных через Apache Hive

In [ ]:
HOST=hive \
new_file /tmp/query.sql <<EOF
USE loader_demo_db;

LOAD DATA INPATH 'hdfs://hadoop:9000/user/hadoop/tab1_hdfs.csv'
INTO TABLE tab1_managed;
EOF

beeline -u 'jdbc:hive2://${HIVE_HOST:-hive}:10000/' -f /tmp/query.sql

### Проверка содержимого таблицы

In [ ]:
impala-shell -i "${IMPALA_HOST}" -d loader_demo_db -q "
SELECT * FROM tab1_managed
"

### Состояние файловой системы

После загрузки файлы пропадают из HDFS:

In [ ]:
hdfs dfs -find /user/hadoop

По факту загруженные файлы _перемещаются_ в директорию базы данных:

In [ ]:
HOST=hive execute \
find /user/hive/warehouse/managed/loader_demo_db.db/tab1_managed

## Загрузка данных по партициям

### Загрузка через внешнюю таблицу

In [ ]:
impala-shell -i "${IMPALA_HOST}" -d loader_demo_db -q "
DROP TABLE IF EXISTS tab_by_year_managed;

CREATE TABLE tab_by_year_managed (
   id INT,
   col_1 BOOLEAN,
   col_2 DOUBLE,
   col_3 TIMESTAMP
)
PARTITIONED BY (year INT);
"

In [ ]:
impala-shell -i "${IMPALA_HOST}" -d loader_demo_db -q "
INSERT INTO tab_by_year_managed PARTITION(year=2012)
SELECT *
  FROM tab1_ext
 WHERE year(col_3) = 2012
"

In [ ]:
impala-shell -i "${IMPALA_HOST}" -d loader_demo_db -q "
SELECT * FROM tab_by_year_managed
"

### Загрузка через Apache Hive

In [ ]:
impala-shell -i "${IMPALA_HOST}" -d loader_demo_db -q "
DROP TABLE IF EXISTS tab_by_year_managed;

CREATE TABLE tab_by_year_managed (
   id INT,
   col_1 BOOLEAN,
   col_2 DOUBLE,
   col_3 TIMESTAMP
)
PARTITIONED BY (year INT)
ROW FORMAT DELIMITED
FIELDS TERMINATED BY ','
STORED AS TEXTFILE
"

### Подготовка внешних файлов

Подготовка файла для 2012 года:

In [ ]:
HOST=${NAMENODE_HOST:-hadoop} \
new_file /tmp/tab_2012_hdfs.csv <<EOF
1,true,123.123,2012-10-24 08:55:00
2,false,1243.5,2012-10-25 13:40:00
EOF

hdfs dfs -mkdir -p /user/hadoop/tab_by_year_managed/year=2012

hdfs dfs -put -f /tmp/tab_2012_hdfs.csv /user/hadoop/tab_by_year_managed/year=2012

hdfs dfs -cat /user/hadoop/tab_by_year_managed/year=2012/tab_2012_hdfs.csv

Подготовка файлов для 2008 года:

In [ ]:
HOST=${NAMENODE_HOST:-hadoop} \
new_file /tmp/tab_2008_june_hdfs.csv <<EOF
3,false,44453.325,2008-06-22 09:33:21.123
4,true,3413423.325,2008-06-12 22:32:21.33454
EOF

hdfs dfs -mkdir -p /user/hadoop/tab_by_year_managed/year=2008

hdfs dfs -put -f /tmp/tab_2008_june_hdfs.csv /user/hadoop/tab_by_year_managed/year=2008

hdfs dfs -cat /user/hadoop/tab_by_year_managed/year=2008/tab_2008_june_hdfs.csv

In [ ]:
HOST=${NAMENODE_HOST:-hadoop} \
new_file /tmp/tab_2008_dec_hdfs.csv <<EOF
5,true,453.325,2008-12-21 19:13:01.123
6,false,1453.325,2008-12-5 02:14:42.33454
EOF

hdfs dfs -mkdir -p /user/hadoop/tab_by_year_managed/year=2008

hdfs dfs -put -f /tmp/tab_2008_dec_hdfs.csv /user/hadoop/tab_by_year_managed/year=2008

hdfs dfs -cat /user/hadoop/tab_by_year_managed/year=2008/tab_2008_dec_hdfs.csv

Таким образом, появились два файла за 2008 год и один файл за 2012 год:

In [ ]:
hdfs dfs -find /user/hadoop/tab_by_year_managed

Таблица `tab_by_year_managed` управляемая, поэтому у нее нет ни одного файла в директории с базой данных:

In [ ]:
HOST=${IMPALA_HOST} execute \
find /user/hive/warehouse/managed/loader_demo_db.db/tab_by_year_managed

In [ ]:
HOST=hive \
new_file /tmp/query.sql <<EOF
USE loader_demo_db;

LOAD DATA INPATH 'hdfs://hadoop:9000/user/hadoop/tab_by_year_managed/year=2012'
OVERWRITE INTO TABLE tab_by_year_managed PARTITION(year=2012);

LOAD DATA INPATH 'hdfs://hadoop:9000/user/hadoop/tab_by_year_managed/year=2008'
INTO TABLE tab_by_year_managed PARTITION(year=2008);
EOF

beeline -u 'jdbc:hive2://${HIVE_HOST:-hive}:10000/' -f /tmp/query.sql

In [ ]:
impala-shell -i "${IMPALA_HOST}" -d loader_demo_db -q "
SELECT * FROM tab_by_year_managed;
"

Загруженные файлы (ожидаемо) пропали с HDFS:

In [ ]:
hdfs dfs -find /user/hadoop/tab_by_year_managed

Загруженные файлы были перемещены в директорию с базой данных:

In [ ]:
HOST=${IMPALA_HOST} execute \
find /user/hive/warehouse/managed/loader_demo_db.db/tab_by_year_managed

## Выводы

1. Apache Impala позволяет загрузить данные из файлов в базу данных при помощи внешней таблицы;
1. Внешняя таблица не владеет файлом, при удалении внешней таблицы, файлы остаются нетронутыми;
1. Apache Hive также можно использовать для загрузки данных из файлов в базу данных;
1. Загружать файлы можно как в обычные таблицы, так и сегментированные;
1. Загрузка в сегментированную таблицу через Apache Hive выполняется по одной партиции за раз;
1. Загрузка через Apache Hive перемещает файлы из источника в директорию базы данных.

### Задания

1. Создать текстовый файл `aston-martin-200x.csv` со списком марок автомобилей Aston Martin, [выпускавшихся с 2001 по 2010 год](https://en.wikipedia.org/wiki/Aston_Martin#Post-war_cars):
    - разделитель: `|`
    - схема: `id, name, year`
1. Создать внешнюю таблицу `cars_ext`, которая загружает файл `aston-martin-200x.csv`;
1. Создать таблицу `cars_managed`:
    - схема: `id, name`
    - сегментирование: `year`
1. Загрузить данные из `cars_ext` в `cars_managed` через `INSERT SELECT`;
1. Загрузить данные из `aston-martin-200x.csv` в `cars_managed` через `LOAD DATA`. Убедиться в отсутствии дубликатов.